### ウェザーデータを獲得する

In [33]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 35.6895,
	"longitude": 139.6917,
	"start_date": "2024-04-01",
	"end_date": "2025-04-01",
	"daily": ["weather_code", "apparent_temperature_mean", "shortwave_radiation_sum", "precipitation_sum", "apparent_temperature_max", "apparent_temperature_min"],
	"timezone": "Asia/Tokyo"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(1).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["weather_code"] = daily_weather_code
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe


Coordinates 35.6766242980957°N 139.6911163330078°E
Elevation 40.0 m asl
Timezone b'Asia/Tokyo'b'GMT+9'
Timezone difference to GMT+0 32400 s


,date,weather_code,apparent_temperature_mean,shortwave_radiation_sum,precipitation_sum,apparent_temperature_max,apparent_temperature_min
0,2024-03-31 15:00:00+00:00,63.0,11.244492,15.070000,4.500000,16.453678,7.874873
1,2024-04-01 15:00:00+00:00,3.0,8.785535,20.480000,0.000000,14.968527,1.242942
2,2024-04-02 15:00:00+00:00,61.0,11.263244,5.850000,13.599999,14.936157,7.407007
3,2024-04-03 15:00:00+00:00,63.0,12.717857,8.400000,7.600000,16.965803,7.456524
4,2024-04-04 15:00:00+00:00,53.0,7.740362,5.140000,3.900000,9.340430,6.009455
...,...,...,...,...,...,...,...
361,2025-03-27 15:00:00+00:00,63.0,19.119020,14.150000,11.200001,27.002457,12.497955
362,2025-03-28 15:00:00+00:00,61.0,4.793403,2.930000,11.900001,9.987745,1.756805
363,2025-03-29 15:00:00+00:00,51.0,5.254597,18.709999,0.700000,11.166651,1.356991
364,2025-03-30 15:00:00+00:00,53.0,2.638526,6.330000,2.200000,3.891933,0.679497


In [34]:
df = pd.read_feather('mergeddf.feather')
df.head(10)

,日付,曜日,来客数,総杯数,売上合計(円),ペールエール(本),ペールエール(円)_x,ラガー(本),ラガー(円)_x,IPA(本),...,フルーツビール(円)_x,平均気温(℃),日照時間(時間),降雪量合計(cm),平均湿度(％),最高気温(℃),最低気温(℃),10分間降水量の最大(mm),天気概況(昼：06時～18時),天気概況(夜：18時～翌日06時)
0,2024-04-01,月,16,25,25300,6,6000,5,4000,5,...,0,14.6,3.8,0.0,73.0,19.7,11.8,1.5,曇時々雨一時晴,快晴
1,2024-04-02,火,19,25,24600,6,6000,4,3200,5,...,1000,14.2,10.6,0.0,50.0,20.6,8.4,0.0,晴後薄曇,曇
2,2024-04-03,水,11,19,18500,5,5000,4,3200,3,...,1000,13.9,0.0,0.0,83.0,16.6,12.3,1.5,雨時々曇,雨時々曇
3,2024-04-04,木,6,10,9500,2,2000,2,1600,2,...,2000,15.4,0.1,0.0,79.0,19.9,11.3,1.5,曇一時雨,曇後一時雨
4,2024-04-05,金,10,18,17100,3,3000,5,4000,3,...,2000,11.4,0.0,0.0,71.0,15.1,9.0,0.5,曇時々雨,曇時々雨
5,2024-04-06,土,17,36,34900,11,11000,8,6400,6,...,1000,12.7,0.0,0.0,81.0,15.8,9.0,0.0,曇,曇時々雨
6,2024-04-08,月,7,9,8600,2,2000,2,1600,1,...,2000,18.8,2.3,0.0,85.0,23.2,15.4,0.0,曇後雨,曇時々雨
7,2024-04-09,火,3,4,4800,1,1000,1,800,1,...,0,15.6,0.0,0.0,78.0,19.8,9.7,5.5,大雨一時曇,曇一時晴
8,2024-04-10,水,8,15,14700,3,3000,3,2400,2,...,3000,11.7,11.9,0.0,47.0,17.8,7.4,0.0,晴,晴一時薄曇
9,2024-04-11,木,15,19,18100,6,6000,4,3200,3,...,2000,14.0,9.3,0.0,62.0,19.8,7.7,0.0,晴後薄曇,曇


In [36]:
# Remove specified columns from df
columns_to_drop = [
    'avg_temperature',
    'hours_of_sunlight',
    'total_snowfall',
    'avg_humidity',
    'max_temperature',
    'min_temperature',
    'max_precipitation_10min',
    'weather_summary_day',
    'weather_summary_night'
]
df = df.drop(columns=columns_to_drop)
df.head()

,date,day_of_week,visitors,total_cups,total_sales_yen,pale_ale_bottles,pale_ale_sales,lager_bottles,lager_sales,ipa_bottles,ipa_sales,white_beer_bottles,white_beer_sales,dark_beer_bottles,dark_beer_sales,fruit_beer_bottles,fruit_beer_sales
0,2024-04-01,月,16,25,25300,6,6000,5,4000,5,4500,4,3600,6,7200,0,0
1,2024-04-02,火,19,25,24600,6,6000,4,3200,5,4500,3,2700,6,7200,1,1000
2,2024-04-03,水,11,19,18500,5,5000,4,3200,3,2700,2,1800,4,4800,1,1000
3,2024-04-04,木,6,10,9500,2,2000,2,1600,2,1800,1,900,1,1200,2,2000
4,2024-04-05,金,10,18,17100,3,3000,5,4000,3,2700,2,1800,3,3600,2,2000


In [41]:
df['date'] = pd.to_datetime(df['date']).dt.date
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date']).dt.date
daily_dataframe['date'] = daily_dataframe['date'] + pd.Timedelta(days=1)
daily_dataframe
# ...existing code...

,date,weather_code,apparent_temperature_mean,shortwave_radiation_sum,precipitation_sum,apparent_temperature_max,apparent_temperature_min
0,2024-04-01,63.0,11.244492,15.070000,4.500000,16.453678,7.874873
1,2024-04-02,3.0,8.785535,20.480000,0.000000,14.968527,1.242942
2,2024-04-03,61.0,11.263244,5.850000,13.599999,14.936157,7.407007
3,2024-04-04,63.0,12.717857,8.400000,7.600000,16.965803,7.456524
4,2024-04-05,53.0,7.740362,5.140000,3.900000,9.340430,6.009455
...,...,...,...,...,...,...,...
361,2025-03-28,63.0,19.119020,14.150000,11.200001,27.002457,12.497955
362,2025-03-29,61.0,4.793403,2.930000,11.900001,9.987745,1.756805
363,2025-03-30,51.0,5.254597,18.709999,0.700000,11.166651,1.356991
364,2025-03-31,53.0,2.638526,6.330000,2.200000,3.891933,0.679497


In [42]:
# Merge df and daily_dataframe on the date column
merged_df = pd.merge(df, daily_dataframe, on='date', how='inner')
merged_df.head()
merged_df.to_feather('newdf.feather')